Расчет обеспеченности озеленением (м2/чел) в каждом типе городской модели на человека. Выводим в difference_from_normative: положительное число - плотность населения в территориальной зоне не превышает норматив, отрицательное - плотность населения в территориальной зоне превышает норматив

In [ ]:
from green_analytics_1 import calculate_green_analytics

# Примените функцию
living_zones = calculate_green_analytics(green, park, living_zones)

vmax = abs(living_zones["difference_from_normative"]).max()
vmin = -vmax

m_5 = living_zones.explore(
    column="difference_from_normative",
    cmap="RdYlGn",
    legend=True,
    style_kwds={"color": "green", "weight": 0.5},
    tiles="CartoDB positron",
    vmin=vmin,
    vmax=vmax
)

m_5

Расчёт плотности населения. Положительное значение - территориальная зона соответсвует нормативу, отрицательное - территориальная зона превышает нормативные показатели плотности

In [ ]:
import geopandas as gpd
from calculate_density import calculate_density

# Проверка и установка CRS
if living_zones.crs is None:
    print("CRS не задан, устанавливаем по умолчанию EPSG:32637...")
    living_zones = living_zones.set_crs(epsg=32637)
else:
    # Преобразуем в нужную СК, если необходимо
    if living_zones.crs.to_epsg() != 32637:
        living_zones = living_zones.to_crs(epsg=32637)

# Расчёт плотности
calculate_density(living_zones)
living_zones


# Визуализация
m_6 = living_zones.explore(
    column="deficit_density",
    cmap="RdYlGn",  # Красный = превышение, зелёный = есть запас
    legend=True,
    style_kwds={
        "color": "green",        # Цвет границы
        "weight": 0.5,           # Толщина границы
        "fillOpacity": 0.7       # Прозрачность заливки
    },
    tooltip=[
        "id_zones",
        "is_living_zones",
        "area_zone",
        "city_model",
        "sum_population",
        "density_population",
        "limit_density",
        "deficit_density"
    ],
    tiles="CartoDB positron"
)

m_6


Анализ распределения на территориях вакантных и занятых мест в обслуживающих сервисах

In [ ]:
from calculating_potential_populating import calculate_and_update

# Применяем расчёты
living_zones = calculate_and_update(living_zones)

# Визуализируем
m_7 = living_zones.explore(
    column="need_dop_service",
    cmap="Set1",
    legend=True,
    style_kwds={"color": "grey", "weight": 0.5},
    tooltip=[
        "id_zones",
        "need_dop_service",
        "new_population",
        "new_population_dop",
    ],
    tiles="CartoDB positron",
)

m_7


Формирование итоговой оценки

In [ ]:
# Формирование оценки при наличии мест в трёх социальных сервисахф

from total_score_new_population import analyze_zones
import geopandas as gpd

# Обрабатываем зоны
zones_0 = analyze_zones(living_zones)

# Визуализация на карте
m_8 = zones_0.explore(
    column="total_score",
    cmap="YlGnBu",
    legend=True,
    tiles="CartoDB positron",
    tooltip=["id_zones", "need_dop_service", "new_population", "total_score", "score_category"],
    style_kwds={"color": "grey", "weight": 0.5, "fillOpacity": 0.65}
)
m_8

In [ ]:
# Формирование оценки при наличии мест в двух из трёх социальных сервисахф

from total_score_new_population_dop import analyze_zones

# Получаем данные из функции analyze_zones
zones_dop = analyze_zones(living_zones)

# Строим интерактивную карту
m_9 = zones_dop.explore(
    column="total_score",
    cmap="YlGnBu",
    legend=True,
    tiles="CartoDB positron",
    tooltip=["id_zones", "need_dop_service", "new_population_dop", "total_score", "score_category"],
    style_kwds={"color": "grey", "weight": 0.5, "fillOpacity": 0.65}
)
m_9

Обрезка вакантных участков города по положительно оцененным участкам.

In [ ]:
from va_cliped_city_model import analyze_zones
import geopandas as gpd

va_path = r"path_to_file" #считываем путь до ранее вычеленного файла с вакантными территориями из Blocksnet
zones_total_score = zones_0.copy() #определение по оценке zones_0


m, va_for_house, va_dop, _ = analyze_zones(zones_total_score, zones, va_path)
va_for_house_0 = va_for_house.copy()
va_dop_0 = va_dop.copy()
m_10 = m
m_10

Дополнительные кварталы

In [ ]:
from va_cliped_city_model import analyze_zones
import geopandas as gpd

va_path = r"path_to_file" #считываем путь до ранее вычеленного файла с вакантными территориями из Blocksnet
zones_total_score = zones_dop.copy() #определение по оценке zones_dop

m, va_for_house, va_dop, _ = analyze_zones(zones_total_score, zones, va_path)
va_for_house_dop = va_for_house.copy()
va_dop_dop = va_dop.copy()

va_for_house_dop = va_for_house_dop.to_crs(epsg=32637)
va_dop_dop = va_dop_dop.to_crs(epsg=32637)

m_11 = m
m_11

Формирование вакантных участков на положительно оцененных территориях

In [ ]:
# Для оценки zones_0

import json

# === 1. НАСТРОЙКИ === #
TYPE_HOUSE_JSON = r"path_to_file" #вставьте путь до house_list.json из примеров
EXCEL_FILE = r"path_to_file" # вставьте путь до exel "normatives_type_house"
ALLOWED_PROPORTION_DEVIATION = 0.9  # Можно менять параметры периметра участков

# === 2. Загрузка JSON === #
with open(TYPE_HOUSE_JSON, "r", encoding="utf-8") as f:
    type_house_data = json.load(f)

# === 3. Подготовка GeoDataFrame === #
va_df = va_for_house_0.copy()  # Убедитесь, что эта переменная определена, либо замените на корректный путь
va_df = va_df.to_crs(epsg=32637)
va_df['area_va'] = va_df.geometry.area

# === 4. Подбор типа дома === #
def select_type_house(city_model, area_va):
    candidates = [
        th for th in type_house_data
        if th["city_model"] == city_model and area_va <= th["max_va_area"]
    ]
    if not candidates:
        return None, None, None, None, None, None

    best_match = sorted(
        candidates,
        key=lambda x: abs(area_va - ((x["min_buff_area"] + x["max_buff_area"]) / 2))
    )[0]

    proportion_str = best_match.get("proportion")
    try:
        if proportion_str:
            p1, p2 = map(float, proportion_str.split(":"))
            proportion_value = p1 / p2
        else:
            proportion_value = None
    except ValueError as e:
        print(f"Ошибка при разборе пропорции: {e}")
        proportion_value = None

    return (
        best_match["type_house"],
        best_match["min_buff_area"],
        best_match["code_house"],
        proportion_value,
        best_match["max_population_house"],
        best_match["max_house_area"]
    )

va_df["selected_type_house"], va_df["min_buff_area"], va_df["code_house"], \
va_df["proportion"], va_df["max_population_house"], va_df["max_house_area"] = zip(*va_df.apply(
    lambda row: select_type_house(row["city_model"], row["area_va"]),
    axis=1
))

# === 5. Базовая фильтрация по типу и min_buff_area === #
va_df_filtered = va_df[
    va_df["selected_type_house"].notnull() & 
    (va_df["area_va"] >= va_df["min_buff_area"])
]

# === 6. Фильтрация по пропорции (учёт отклонения) === #
def is_proportion_valid(geom, target_ratio):
    if pd.isna(target_ratio):
        return False
    bounds = geom.bounds
    width = abs(bounds[2] - bounds[0])
    height = abs(bounds[3] - bounds[1])
    actual_ratio = width / height if height != 0 else 0
    lower = target_ratio * (1 - ALLOWED_PROPORTION_DEVIATION)
    upper = target_ratio * (1 + ALLOWED_PROPORTION_DEVIATION)
    return lower <= actual_ratio <= upper

va_df_filtered = va_df_filtered[
    va_df_filtered.apply(lambda row: is_proportion_valid(row.geometry, row.proportion), axis=1)
]

# === 7. Статистика === #
print(f"✅ Готово! Отобрано {len(va_df_filtered)} участков из {len(va_df)}")

initial_counts = va_df.groupby('city_model').size()
filtered_counts = va_df_filtered.groupby('city_model').size()

for city_model in initial_counts.index:
    print(f"City model: {city_model} — До: {initial_counts[city_model]}, После: {filtered_counts.get(city_model, 0)}")

type_house_counts = va_df_filtered["selected_type_house"].value_counts()
type_house_pct = va_df_filtered["selected_type_house"].value_counts(normalize=True) * 100

print("\n📊 Распределение по типам домов:")
for th, count in type_house_counts.items():
    pct = type_house_pct.get(th, 0)
    print(f"  - {th}: {count} участков ({pct:.2f}%)")

# === 8. Слияние с Excel === #
gdf = va_df_filtered.copy()
gdf.rename(columns={'CodeHouse': 'code_house'}, inplace=True)
gdf['code_house'] = gdf['code_house'].astype(str)

excel_df = pd.read_excel(EXCEL_FILE)
excel_df.rename(columns={'CodeHouse': 'code_house'}, inplace=True)
excel_df['code_house'] = excel_df['code_house'].astype(str)

gdf = gdf.merge(excel_df, on='code_house', how='left')
gdf = gpd.GeoDataFrame(gdf, geometry='geometry')

# Переименование колонок
gdf = gdf.rename(columns={
    'sum_population': 'Количество проживающего населения на территории',
    'new_population_dop': 'Потенциальное население при условии размещения дополнительного сервиса',
    'need_dop_service': 'Тип дополнительного сервиса',
    'total_score': 'Оценка потенциала территории для интенсификации',
    'area_va': 'Площадь вакантного участка (м2)'
})

# === 9. Визуализация === #
import folium
m_12 = gdf.explore(
    column="selected_type_house",
    legend=True,
    tiles="CartoDB positron",
    location=[57.6261, 39.8845],  # центр Ярославля
    zoom_start=12
)

va_df_filtered_0 = va_df_filtered
select_va_0 = gdf

m_12  # В Jupyter используйте "m" для отображения карты


In [ ]:
# Для оценки zones_dop

# === 1. НАСТРОЙКИ === #
TYPE_HOUSE_JSON = r"path_to_file" #вставьте путь до house_list.json из примеров
EXCEL_FILE = r"path_to_file" # вставьте путь до exel "normatives_type_house"
ALLOWED_PROPORTION_DEVIATION = 0.9  # Можно менять параметры периметра участков

# === 2. Загрузка JSON === #
with open(TYPE_HOUSE_JSON, "r", encoding="utf-8") as f:
    type_house_data = json.load(f)

# === 3. Подготовка GeoDataFrame === #
va_df = va_for_house_dop.copy()  # Убедитесь, что эта переменная определена, либо замените на корректный путь
va_df = va_df.to_crs(epsg=32637)
va_df['area_va'] = va_df.geometry.area

# === 4. Подбор типа дома === #
def select_type_house(city_model, area_va):
    candidates = [
        th for th in type_house_data
        if th["city_model"] == city_model and area_va <= th["max_va_area"]
    ]
    if not candidates:
        return None, None, None, None, None, None

    best_match = sorted(
        candidates,
        key=lambda x: abs(area_va - ((x["min_buff_area"] + x["max_buff_area"]) / 2))
    )[0]

    proportion_str = best_match.get("proportion")
    try:
        if proportion_str:
            p1, p2 = map(float, proportion_str.split(":"))
            proportion_value = p1 / p2
        else:
            proportion_value = None
    except ValueError as e:
        print(f"Ошибка при разборе пропорции: {e}")
        proportion_value = None

    return (
        best_match["type_house"],
        best_match["min_buff_area"],
        best_match["code_house"],
        proportion_value,
        best_match["max_population_house"],
        best_match["max_house_area"]
    )

va_df["selected_type_house"], va_df["min_buff_area"], va_df["code_house"], \
va_df["proportion"], va_df["max_population_house"], va_df["max_house_area"] = zip(*va_df.apply(
    lambda row: select_type_house(row["city_model"], row["area_va"]),
    axis=1
))

# === 5. Базовая фильтрация по типу и min_buff_area === #
va_df_filtered = va_df[
    va_df["selected_type_house"].notnull() & 
    (va_df["area_va"] >= va_df["min_buff_area"])
]

# === 6. Фильтрация по пропорции (учёт отклонения) === #
def is_proportion_valid(geom, target_ratio):
    if pd.isna(target_ratio):
        return False
    bounds = geom.bounds
    width = abs(bounds[2] - bounds[0])
    height = abs(bounds[3] - bounds[1])
    actual_ratio = width / height if height != 0 else 0
    lower = target_ratio * (1 - ALLOWED_PROPORTION_DEVIATION)
    upper = target_ratio * (1 + ALLOWED_PROPORTION_DEVIATION)
    return lower <= actual_ratio <= upper

va_df_filtered = va_df_filtered[
    va_df_filtered.apply(lambda row: is_proportion_valid(row.geometry, row.proportion), axis=1)
]

# === 7. Статистика === #
print(f"✅ Готово! Отобрано {len(va_df_filtered)} участков из {len(va_df)}")

initial_counts = va_df.groupby('city_model').size()
filtered_counts = va_df_filtered.groupby('city_model').size()

for city_model in initial_counts.index:
    print(f"City model: {city_model} — До: {initial_counts[city_model]}, После: {filtered_counts.get(city_model, 0)}")

type_house_counts = va_df_filtered["selected_type_house"].value_counts()
type_house_pct = va_df_filtered["selected_type_house"].value_counts(normalize=True) * 100

print("\n📊 Распределение по типам домов:")
for th, count in type_house_counts.items():
    pct = type_house_pct.get(th, 0)
    print(f"  - {th}: {count} участков ({pct:.2f}%)")

# === 8. Слияние с Excel === #
gdf = va_df_filtered.copy()
gdf.rename(columns={'CodeHouse': 'code_house'}, inplace=True)
gdf['code_house'] = gdf['code_house'].astype(str)

excel_df = pd.read_excel(EXCEL_FILE)
excel_df.rename(columns={'CodeHouse': 'code_house'}, inplace=True)
excel_df['code_house'] = excel_df['code_house'].astype(str)

gdf = gdf.merge(excel_df, on='code_house', how='left')
gdf = gpd.GeoDataFrame(gdf, geometry='geometry')

# Переименование колонок
gdf = gdf.rename(columns={
    'sum_population': 'Количество проживающего населения на территории',
    'new_population_dop': 'Потенциальное население при условии размещения дополнительного сервиса',
    'need_dop_service': 'Тип дополнительного сервиса',
    'total_score': 'Оценка потенциала территории для интенсификации',
    'area_va': 'Площадь вакантного участка (м2)'
})

# === 9. Визуализация === #
import folium
m_13 = gdf.explore(
    column="selected_type_house",
    legend=True,
    tiles="CartoDB positron",
    location=[57.6261, 39.8845],  # центр Ярославля
    zoom_start=12
)
va_df_filtered_dop = va_df_filtered

select_va__dop = gdf
m_13  # В Jupyter используйте "m" для отображения карты


In [ ]:
# Сохранение итоговых геослоев

zones_0.to_file(r"path_to_file")
zones_dop.to_file(r"path_to_file")
select_va_0.to_file(r"path_to_file")
select_va__dop.to_file(r"path_to_file")

In [ ]:
# Сохранение html карт на каждом этапе

maps = [
    (m_1, "city_model.html"),
    (m_5, "difference_from_normative_green.html"),
    (m_6, "calculate_density_population.html"),
    (m_7, "calculating_potential_populating.html"),
    (m_8, "total_score_new_population1.html"),
    (m_9, "total_score_new_population_dop1.html"),
    (m_10, "va_cliled_visualization_01.html"),
    (m_11, "va_cliled_visualization_dop1.html"),
    (m_12, "selected_type_house_01.html"),
    (m_13, "selected_type_house_dop1.html")
]

output_folder = r"path_to_folder" #путь до папки сохранения

for m, filename in maps:
    m.save(f"{output_folder}\\{filename}")


In [ ]:
# Сохранение скриншотов карт

import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

def capture_screenshots_from_html(input_folder_path, output_folder_path):
    # Настройка браузера
    options = Options()
    options.add_argument("--headless")  # Без графического интерфейса
    options.add_argument("--window-size=1920,1200")  # Размер экрана
    driver = webdriver.Chrome(options=options)

    # Проверка существования папки для сохранения картинок
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)  # Создаем папку, если она не существует

    # Проходим по всем HTML файлам в указанной папке
    for file_name in os.listdir(input_folder_path):
        if file_name.endswith(".html"):  # Проверяем только HTML файлы
            html_path = os.path.join(input_folder_path, file_name)
            driver.get("file://" + os.path.abspath(html_path))  # Открываем HTML файл

            time.sleep(2)  # Ждем, чтобы страница прогрузилась

            # Выполняем увеличение масштаба (если кнопка зума доступна)
            driver.execute_script("""
                var zoomInButton = document.querySelector('.leaflet-control-zoom-in');
                if (zoomInButton) {
                    zoomInButton.click();  // Кликаем по кнопке увеличения зума
                }
            """)

            time.sleep(2)  # Ждем, чтобы зум применился

            # Создаем имя файла для скриншота
            screenshot_name = file_name.replace(".html", ".jpg")
            screenshot_path = os.path.join(output_folder_path, screenshot_name)

            # Сохраняем скриншот
            driver.save_screenshot(screenshot_path)
            print(f"Скриншот сохранен: {screenshot_path}")

    driver.quit()

# Пример использования
input_folder = r"path_to_folder"  # Папка с HTML файлами
output_folder = r"path_to_folder"  # Папка для сохранения скриншотов

capture_screenshots_from_html(input_folder, output_folder)
